In [5]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras import layers
import tensorflow as tf


In [9]:
# 필터 시각화
model = keras.applications.xception.Xception(
    weights="imagenet",
    include_top=False)

layer_name = ['block1_conv1', 'block1_conv2', 'block2_sepconv1', 'block2_sepconv2' ,'block3_sepconv1','block3_sepconv2' ]
for layer_name in layer_name:
    layer = model.get_layer(name=layer_name)
    feature_extractor = keras.Model(inputs=model.input, outputs=layer.output)


    def compute_loss(image, filter_index):
        activation = feature_extractor(image)
        filter_activation = activation[:, 2:-2, 2:-2, filter_index]
        return tf.reduce_mean(filter_activation)
    @tf.function
    def gradient_ascent_step(image, filter_index, learning_rate):
        with tf.GradientTape() as tape:
            tape.watch(image)
            loss = compute_loss(image, filter_index)
        grads = tape.gradient(loss, image)
        grads = tf.math.l2_normalize(grads)
        image += learning_rate * grads
        return image
    img_width = 200
    img_height = 200

    def generate_filter_pattern(filter_index):
        iterations = 30
        learning_rate = 10.
        image = tf.random.uniform(
            minval=0.4,
            maxval=0.6,
            shape=(1, img_width, img_height, 3))
        for i in range(iterations):
            image = gradient_ascent_step(image, filter_index, learning_rate)
        return image[0].numpy()
    
    def deprocess_image(image):
        image -= image.mean()
        image /= image.std()
        image *= 64
        image += 128
        image = np.clip(image, 0, 255).astype("uint8")
        image = image[25:-25, 25:-25, :]
        return image
    
    keras.utils.save_img(f"filters_for_layer_{layer_name}.png", deprocess_image(generate_filter_pattern(filter_index=2)))